In [216]:
from pathlib import Path
from collections import deque
from math import prod

def load_file(path):
    res = {}
    with Path(path).open() as f:
        for line in f.readlines():
            mod, dest = line.split(" -> ")
            if mod == "broadcaster":
                mod_type = mod_name = "broadcaster"
            else:
                mod_type, mod_name = mod[0], mod[1:]
            res[mod_name] = (mod_type, [], [d.strip() for d in dest.split(",")])

    for mod_name, config in res.items():
        for dest in config[2]:
            if dest in res:
                res[dest][1].append(mod_name)
    return res

LOW = 0
HIGH = 1

class Broadcaster:

    def __init__(self, name, sources, dests):
        self.name = name
        self.dests = dests
        self.sources = sources
    
    def input(self, signal, source):
        res = []
        for dest in self.dests:
            res.append((signal, self.name, dest))
        return res

    def __repr__(self):
        return f'[{", ".join(self.sources)}] -> Broadcast -> [{", ".join(self.dests)}]'

class FlipFlop:
    
    def __init__(self, name, sources, dests):
        self.name = name
        self.dests = dests
        self.is_on = False
        self.sources = sources
    
    def input(self, signal, source):
        res =  []
        if signal == LOW:
            self.is_on = not self.is_on
            signal = HIGH if self.is_on else LOW
            for dest in self.dests:
                res.append((signal, self.name, dest))
        return res

    def __repr__(self):
        return f'[{", ".join(self.sources)}] -> FlipFlop -> [{", ".join(self.dests)}]'
                
class Conjunction:
    def __init__(self, name, sources, dests):
        self.name = name
        self.sources = {s: LOW for s in sources}
        self.dests = dests

    def input(self, signal, source):
        res = []
        self.sources[source] = signal
        signal = LOW if all(self.sources.values()) else HIGH
        for dest in self.dests:
            res.append((signal, self.name, dest))
        return res

    def __repr__(self):
        return f'[{", ".join(self.sources)}] -> Conjunction -> [{", ".join(self.dests)}]'
                

constructors = {
    "broadcaster": Broadcaster,
    "%": FlipFlop,
    "&": Conjunction
}

def create_modules(schematics):
    modules = {}
    for mod_name, config in schematics.items():
        modules[mod_name] = constructors[config[0]](mod_name, config[1], config[2])
    return modules

def push_button(modules):
    pulses = [1, 0]
    queue = deque(modules["broadcaster"].input(LOW, "broadcaster"))
    while queue:
        sig, source, dest = queue.popleft()
        pulses[sig] += 1
        # print(sig, source, dest)
        if dest in modules:
            queue.extend(modules[dest].input(sig, source))
    return pulses

def solve1(path):
    schematics = load_file(path)
    modules = create_modules(schematics)
    res = [0, 0]
    for _ in range(1000):
        tmp = push_button(modules)
        res = [res[0] + tmp[0], res[1] + tmp[1]]
    return prod(res)

def solve2(path):
    schematics = load_file(path)
    modules = create_modules(schematics)
    pushes = 0
    inputs = {}
    while True:
        pushes += 1
        queue = deque(modules["broadcaster"].input(LOW, "broadcaster"))
        while queue:
            sig, source, dest = queue.popleft()
            if sig == HIGH:
                if source in ["ks", "pm", "dl", "vk"]:
                    if source not in inputs:
                        inputs[source] = pushes
                    else:
                        return inputs
            if dest in modules:
                queue.extend(modules[dest].input(sig, source))

In [217]:
schematics = load_file("20_test.txt")
schematics

{'broadcaster': ('broadcaster', [], ['a', 'b', 'c']),
 'a': ('%', ['broadcaster', 'inv'], ['b']),
 'b': ('%', ['broadcaster', 'a'], ['c']),
 'c': ('%', ['broadcaster', 'b'], ['inv']),
 'inv': ('&', ['c'], ['a'])}

In [218]:
modules = create_modules(schematics)
modules

{'broadcaster': [] -> Broadcast -> [a, b, c],
 'a': [broadcaster, inv] -> FlipFlop -> [b],
 'b': [broadcaster, a] -> FlipFlop -> [c],
 'c': [broadcaster, b] -> FlipFlop -> [inv],
 'inv': [c] -> Conjunction -> [a]}

In [219]:
push_button(modules)

[8, 4]

In [220]:
solve1("20_test.txt") == 32000000

True

In [221]:
solve1("20_test2.txt") == 11687500

True

In [222]:
solve1("20_input.txt")  # 711650489

711650489

In [224]:
from math import lcm
lcm(*solve2("20_input.txt").values())  # 219388737656593

219388737656593